# Weights Distances

In [1]:
######################## (70m) ############################

In [2]:
from tqdm import tqdm
from transformers import GPTNeoXForCausalLM, AutoTokenizer
import concurrent.futures
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns



tokenizer = AutoTokenizer.from_pretrained( "EleutherAI/pythia-70m-deduped", revision="step143000" )
ref_model = GPTNeoXForCausalLM.from_pretrained( "EleutherAI/pythia-70m-deduped", revision="step143000")
ref_model_params = ref_model.state_dict()

In [3]:
# Define the function to calculate layer distances
def calculate_layer_distances(step_idx,rev_id):
    model = GPTNeoXForCausalLM.from_pretrained( "EleutherAI/pythia-70m-deduped", revision=rev_id)
    model_params = model.state_dict()
        
    layer_dists = {}
    
    for layer in list(model_params.keys()):
        # Extract weights for the current layer
        weights_model = model_params[layer]
        weights_ref_model = ref_model_params[layer]

        # Calculate Euclidean distance (you can replace this with other metrics)
        euclidean_distance = (weights_model - weights_ref_model).norm().item()
        layer_dists[layer] = euclidean_distance
        print(f"Step {rev_id} Layer {layer}: Euclidean Distance = {euclidean_distance}")

    del model
    return layer_dists

In [8]:
# Define the function to calculate layer distances
def calculate_layer_diff_rank(step_idx,rev_id):
    model = GPTNeoXForCausalLM.from_pretrained( "EleutherAI/pythia-70m-deduped", revision=rev_id)
    model_params = model.state_dict()
        
    rank_dict = {}
    
    for layer in list(model_params.keys()):
        # Extract weights for the current layer
        weights_model = model_params[layer]
        weights_ref_model = ref_model_params[layer]

        # Calculate Euclidean distance (you can replace this with other metrics)
        rank = np.linalg.matrix_rank(weights_ref_model.cpu().detach().numpy()-weights_model.cpu().detach().numpy())

        # Store the rank in the dictionary
        rank_dict[layer] = rank
        print(f"Step {rev_id} Layer {layer}: Rank = {rank}")

    del model
    return rank_dict

In [9]:
# Set the number of steps and the number of steps to process simultaneously
num_steps = 143  # Adjust this according to your actual number of steps

In [10]:
# since the distance is majorly in the first and last layers, should we apply layerwise learning rates ?3
# the overall "loss" is sum of individial layer-losses
# so, from the plots of layerwise distances it can be seen which layer has a steep-decrease in distance and which layer does'nt 
# this can be used for layerwise finetuning ? 

In [11]:

weight_diff_ranks = {}

# Iterate over the steps without using ThreadPoolExecutor
for step_idx in tqdm(range(num_steps), total=num_steps):
    revision_id = f"step{1000*(step_idx+1)}"
    weight_diff_ranks[step_idx] = calculate_layer_diff_rank(step_idx, revision_id)


  0%|                                                                                                                                                                                      | 0/143 [00:00<?, ?it/s]

Step step1000 Layer gpt_neox.embed_in.weight: Rank = 512
Step step1000 Layer gpt_neox.layers.0.input_layernorm.weight: Rank = 1
Step step1000 Layer gpt_neox.layers.0.input_layernorm.bias: Rank = 1
Step step1000 Layer gpt_neox.layers.0.post_attention_layernorm.weight: Rank = 1
Step step1000 Layer gpt_neox.layers.0.post_attention_layernorm.bias: Rank = 1
Step step1000 Layer gpt_neox.layers.0.attention.query_key_value.weight: Rank = 512
Step step1000 Layer gpt_neox.layers.0.attention.query_key_value.bias: Rank = 1
Step step1000 Layer gpt_neox.layers.0.attention.dense.weight: Rank = 511
Step step1000 Layer gpt_neox.layers.0.attention.dense.bias: Rank = 1
Step step1000 Layer gpt_neox.layers.0.mlp.dense_h_to_4h.weight: Rank = 512
Step step1000 Layer gpt_neox.layers.0.mlp.dense_h_to_4h.bias: Rank = 1
Step step1000 Layer gpt_neox.layers.0.mlp.dense_4h_to_h.weight: Rank = 512
Step step1000 Layer gpt_neox.layers.0.mlp.dense_4h_to_h.bias: Rank = 1
Step step1000 Layer gpt_neox.layers.1.input_layer

  1%|█▏                                                                                                                                                                            | 1/143 [00:04<09:39,  4.08s/it]

Step step1000 Layer embed_out.weight: Rank = 512
Step step2000 Layer gpt_neox.embed_in.weight: Rank = 512
Step step2000 Layer gpt_neox.layers.0.input_layernorm.weight: Rank = 1
Step step2000 Layer gpt_neox.layers.0.input_layernorm.bias: Rank = 1
Step step2000 Layer gpt_neox.layers.0.post_attention_layernorm.weight: Rank = 1
Step step2000 Layer gpt_neox.layers.0.post_attention_layernorm.bias: Rank = 1
Step step2000 Layer gpt_neox.layers.0.attention.query_key_value.weight: Rank = 512
Step step2000 Layer gpt_neox.layers.0.attention.query_key_value.bias: Rank = 1
Step step2000 Layer gpt_neox.layers.0.attention.dense.weight: Rank = 511
Step step2000 Layer gpt_neox.layers.0.attention.dense.bias: Rank = 1
Step step2000 Layer gpt_neox.layers.0.mlp.dense_h_to_4h.weight: Rank = 512
Step step2000 Layer gpt_neox.layers.0.mlp.dense_h_to_4h.bias: Rank = 1
Step step2000 Layer gpt_neox.layers.0.mlp.dense_4h_to_h.weight: Rank = 512
Step step2000 Layer gpt_neox.layers.0.mlp.dense_4h_to_h.bias: Rank = 1


  1%|██▍                                                                                                                                                                           | 2/143 [00:08<10:07,  4.31s/it]

...

Step step24000 Layer embed_out.weight: Rank = 512
Step step25000 Layer gpt_neox.embed_in.weight: Rank = 512
Step step25000 Layer gpt_neox.layers.0.input_layernorm.weight: Rank = 1
Step step25000 Layer gpt_neox.layers.0.input_layernorm.bias: Rank = 1
Step step25000 Layer gpt_neox.layers.0.post_attention_layernorm.weight: Rank = 1
Step step25000 Layer gpt_neox.layers.0.post_attention_layernorm.bias: Rank = 1
Step step25000 Layer gpt_neox.layers.0.attention.query_key_value.weight: Rank = 512
Step step25000 Layer gpt_neox.layers.0.attention.query_key_value.bias: Rank = 1
Step step25000 Layer gpt_neox.layers.0.attention.dense.weight: Rank = 512
Step step25000 Layer gpt_neox.layers.0.attention.dense.bias: Rank = 1
Step step25000 Layer gpt_neox.layers.0.mlp.dense_h_to_4h.weight: Rank = 512
Step step25000 Layer gpt_neox.layers.0.mlp.dense_h_to_4h.bias: Rank = 1
Step step25000 Layer gpt_neox.layers.0.mlp.dense_4h_to_h.weight: Rank = 512
Step step25000 Layer gpt_neox.layers.0.mlp.dense_4h_to_h.b

 17%|██████████████████████████████▏                                                                                                                                              | 25/143 [01:53<08:48,  4.48s/it]

Step step25000 Layer embed_out.weight: Rank = 512


 17%|██████████████████████████████▏                                                                                                                                              | 25/143 [01:54<09:01,  4.59s/it]


KeyboardInterrupt: 

In [ ]:
with open('pythia_70m_layer_diff_rank.pickle', 'wb') as file:
    pickle.dump(step_dists, file)

In [ ]:
# Distance form the last-epoch's layer's weights vs current-epcoh's

In [ ]:
plot_data = []
for epoch, layers in step_dists.items():
    for layer, distance in layers.items():
        plot_data.append({'Epoch': int(epoch), 'Layer': layer, 'Distance': distance})

# Creating a DataFrame from the structured data
df = pd.DataFrame(plot_data)

# Sorting the DataFrame by Epoch for better plotting
df.sort_values(by='Epoch', inplace=True)

# Setting up the plot style
sns.set(style="whitegrid")

# Creating the plot
plt.figure(figsize=(30, 16))

# Plotting a line for each layer
for layer in list(ref_model_params.keys()):
    if "rotary_emb.inv_freq" not in layer:
        sns.lineplot(data=df[df['Layer'] == layer], x='Epoch', y='Distance', label=layer)

# Adding plot labels and title
plt.xlabel('Epoch')
plt.ylabel('Distance')
plt.title('Distance by Layer over Different Epochs')
plt.legend(title='Layers', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
# Distance form the last-epoch's layer's weights vs current-epcoh's (log-scale)

In [ ]:

# Setting up the plot style
sns.set(style="whitegrid")

# Creating the plot
plt.figure(figsize=(30, 16))

# Dictionary to store slope data for each layer
slope_data = {'Layer': [], 'Slope': []}

# Plotting a line for each layer
for layer in list(ref_model_params.keys()):
    if ("rotary_emb.inv_freq" not in layer) and ("bias" not in layer):
        layer_data = df[df['Layer'] == layer]
        sns.lineplot(data=layer_data, x='Epoch', y='Distance', label=layer)

        # Calculating the slope using linear regression
        x = layer_data['Epoch']
        y = layer_data['Distance']
        slope, intercept = np.polyfit(x, y, 1)

        # Adding slope data to the dictionary
        slope_data['Layer'].append(layer)
        slope_data['Slope'].append(slope)

# Adding plot labels and title
plt.xlabel('Epoch')
plt.ylabel('Distance (log scale)')
plt.yscale('log')  # Set y-axis to logarithmic scale
plt.title('Distance by Layer over Different Epochs')
plt.legend(title='Layers', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
# Creating a DataFrame for the slope data
slope_df = pd.DataFrame(slope_data)

# Displaying the slope data
print("Slope Data:")
print(slope_df)